# TO DO:
* Figure out how to parse pages with separate project details sections, and with multiple units on different lines within a project details section
  * Example: https://www.gem.wiki/Qiaotou_power_station

# compare working spreadsheet against wiki (not finalized spreadsheet)

In [ ]:
import pandas as pd
import time

In [ ]:
# import urllib.parse
from urllib.parse import unquote
# from os import listdir
# from os.path import isfile, join

In [ ]:
import fuzzywuzzy as fuzz
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Import spreadsheet data (working version)

In [ ]:
gcpt_file = 'Global Coal Plant Tracker Files 2021-12-09_1427 - compiled.xlsx'    
gcpt = pd.read_excel(
    gcpt_path + gcpt_file, 
    sheet_name='Sheet1', 
    dtype={'Wiki page': str}
)

In [ ]:
# TO DO: delete if covered by cell 2 below
# # create column 'Wiki title' in the df

# # TO DO: need to also convert special characters (such as accented letters)
# # (see code in Latin America Portal map data compiling 2021-11-02.ipynb)

# # TO DO: add step to compare, within the spreadsheet, the wiki page url/title and the plant name
# # Some mismatches may be due to old URLs that now redirect

# wiki_str = gcpt['Wiki page']
# wiki_str = wiki_str.str.replace('https://gem.wiki/', 'https://www.gem.wiki/', regex=False)
# wiki_str = wiki_str.str.split('https://www.gem.wiki/')
# wiki_str = wiki_str.str[-1].str.replace('_', ' ')
# gcpt['Wiki title'] = wiki_str

In [ ]:
# TO DO: delete this cell?

# titles_to_process = df['title'].unique().tolist()

# # exclude template pages
# titles_to_process = [x for x in titles_to_process if x.endswith('template page') == False]

# # clean up names
# titles_to_process = [x.replace('&amp;', '&') for x in titles_to_process]

In [ ]:
# create wiki page titles from URLs
spreadsheet = gcpt.copy()
spreadsheet['Wiki page'] = spreadsheet['Wiki page'].astype(str)

unexpected_count = 0 # initialize
for row in spreadsheet.index:
    wiki = spreadsheet.at[row, 'Wiki page']
    plant = spreadsheet.at[row, 'Plant']

    if wiki.startswith('http') and 'gem.wiki' in wiki:
        # convert wiki urls to wiki page names
        w = unquote(wiki)
        w = w.replace('_', ' ')
        w = w.split('gem.wiki/')[-1]
        
        # put value into spreadsheet
        spreadsheet.at[row, 'Wiki title'] = w
    else:
        # print('Error!' + f" Unexpected format for wiki URL for {plant}: {wiki}")
        print(f"{plant}: {wiki}")
        unexpected_count += 1
        
print(unexpected_count)

In [ ]:
# # export wiki titles to excel
# # note that some of the rows don't have a wiki title (perhaps because of links above to sites other than GEM)

# wiki_titles_file_name = gcpt_file.replace('compiled.xlsx', 'wiki titles.xlsx')
# spreadsheet['Wiki title'].sort_values().to_excel(wiki_titles_file_name, index=False)

In [ ]:
# rename columns to fit with wiki pages:
spreadsheet = spreadsheet.rename(columns={'Year': 'Start year'})

In [ ]:
spreadsheet[spreadsheet['Plant'].str.contains('  ')]['Plant'].drop_duplicates()

In [ ]:
# sandbox:
test = spreadsheet[spreadsheet['Wiki page']=='nan']['Plant'].drop_duplicates()
for row in test.index:
    print(test.at[row])

In [ ]:
# sandbox: how often are there more than one set of coordinates for a given plant?
unit_count = spreadsheet.groupby('Plant')['Plant'].count()
unit_multi = unit_count[unit_count > 1]
num_plants_multi_unit = len((unit_multi.index.unique()))

lat_count = spreadsheet[['Plant', 'Latitude']].drop_duplicates().groupby('Plant')['Latitude'].count()

lat_multi = lat_count[lat_count > 1]
print(f"Number of plants with more than one set of coordinates: {len(lat_multi)} (of {num_plants_multi_unit} plants with more than one unit).")

# show:
spreadsheet[spreadsheet['Plant'].isin(lat_multi.index)].sort_values(by=['Plant', 'Unit'])[['Plant', 'Unit', 'Latitude', 'Longitude']].head(20)

## Wiki XML file

In [ ]:
# # define wiki file (XML)

# # china_file = 'GEM wiki export - Coal plants in China - 20211104105337.xml'
# china_file = 'GEM wiki export - Existing coal plants in China - 20211104131345.xml'
# china_path = '/Users/masoninman/Downloads/'

xml_file = 'GEM wiki export 2021-12-09_190304 - coal plants (all in working spreadsheet).xml'

In [ ]:
# WIKI: read in whole XML file
with open(gcpt_path + xml_file) as f: data = f.readlines()

In [ ]:
line_counter = 0 # initialize
page_line_numbers = []
for line in data:
    if line=='  <page>\n':
        page_line_numbers += [line_counter]
    line_counter += 1
    
line_counter = 0 # initialize
end_page_line_numbers = []
for line in data:
    if line=='  </page>\n':
        end_page_line_numbers += [line_counter]
    line_counter += 1
    
print(f"Number of page lines: {len(page_line_numbers)}")

all_pages = []
if len(page_line_numbers) == len(end_page_line_numbers):
    for num in range(len(page_line_numbers)):
        # get the line numbers for start and end of page
        range_of_lines_to_group = range(
            page_line_numbers[num], 
            end_page_line_numbers[num]+1)

        page_strs = []
        for line in range_of_lines_to_group:
            page_strs += [data[line]]
            
        all_pages += [page_strs]

In [ ]:
def find_redirects(all_pages):
    redirects_list = [] # initialize
    for page_num in range(len(all_pages)):
        page_list = all_pages[page_num]
        for line in page_list:
            title = '' # initialize
            redirect_title = line.replace('<redirect title=', '').replace(' />', '').strip('/n').strip().strip('"')
            if '<redirect title' in line:
                # get title line also:
                for other_line in page_list:
                    if '<title>' in other_line:
                        title = other_line.replace('<title>', '').replace('</title>', '').strip('/n').strip()
                    else:
                        pass

                redirects_list += [(title, redirect_title)]
            else:
                # no redirect
                pass

    redirects_df = pd.DataFrame(redirects_list, columns=['Wiki title', 'Wiki redirect title'])
    return redirects_df

redirects_df = find_redirects(all_pages)

redirects_df

In [ ]:
def mismatches_plant_name_and_wiki_title(spreadsheet, redirects_df):
    """
    Uses redirects established above to see what the final wiki page title is.
    """

    test = spreadsheet.copy()[['Plant', 'Wiki title']].drop_duplicates()

    # merge in redirects
    test = pd.merge(
        test,
        redirects_df,
        on='Wiki title', 
        how='left')

    for row in test.index:
        if pd.isna(test.at[row, 'Wiki redirect title']):
            test.at[row, 'Wiki redirect title'] = test.at[row, 'Wiki title']

    test = test.rename(columns={'Wiki redirect title': 'Wiki title (after redirects)'})

    # for this first comparison, ignore capitalization
    test = test[
        test['Wiki title (after redirects)'].str.strip().str.lower()!=
        test['Plant'].str.strip().str.lower()
    ][['Plant', 'Wiki title (after redirects)']]
    
    test = test.sort_values(by=['Plant', 'Wiki title (after redirects)'])
    test = test.reset_index(drop=True)
    
    # exclude those with missing wiki title
    test = test[test['Wiki title (after redirects)'].isna()==False]
    
    return test

test = mismatches_plant_name_and_wiki_title(spreadsheet, redirects_df)
# test[test['Wiki title (after redirects)'].str.contains('redirect', na=False)].reset_index(drop=True)

len(test)

In [ ]:
# sandbox:
# find plants with more than one sponsor
test = spreadsheet.copy()
test = test[['Plant', 'Sponsor']].drop_duplicates()
sponsor_count = test.groupby('Plant')['Sponsor'].count()
sponsor_multi = sponsor_count[sponsor_count > 1]

spreadsheet[spreadsheet['Plant'].isin(sponsor_multi.index)][['Plant', 'Unit', 'Status', 'Sponsor', 'Parent']].sort_values(by=['Plant', 'Unit']).reset_index(drop=True).loc[61:90]

In [ ]:
# spreadsheet[spreadsheet['Plant'].str.contains('Blachownia power station')].sort_values(by='Unit')[['Plant', 'Unit', 'Status', 'Capacity (MW)', 'Sponsor', 'Parent']]

In [ ]:
# Examples:
# Afşin-Elbistan-A power station
# wiki page is Afşin-Elbistan power complex, which includes plants A, B, C
# ownership for Afşin-Elbistan A described in narrative section, but there's no mention of any units in plant A being owned by EÜAŞ
# Project Details for Afşin-Elbistan B

# Asam-Asam power station
# Operating units have ownership in narrative section; project details is only for units under construction

# Bander power station
# Ownership for cancelled unit 3 is not included in wiki page

# Banten Suralaya power station:
# only project details section is "Project Details for Units 9 & 10 (1000MW) Expansion"
# background section has details on operating units, and says "The complex is owned by PT PLN." 
# That agrees with spreadsheet parent, but spreadsheet sponsor isn't included in wiki.

# Bau-Bau Power Station
# Undetermined. Similar but not exactly the same entries in wiki vs spreadsheet

# Beilun power station
# Wiki page has complicated entry for Sponsor:
# Sponsor: Beijing Guodian Power Co,. Ltd. 北京国电电力有限公司 70% of unit 1&2, 49% of unit 3-5, 50% of unit 6 &7; Zhejiang Zheneng Power Co,. Ltd. 浙江浙能电力股份有限公司 30% of unit 1&2, 51% of unit 3-5, 40% of unit 6&7; Ningbo Thermal Power Co., Ltd.宁波热电股份有限公司 10% of unit 6&7.
# Parent entry seems to be just the single controlling parent of each sponsor 
# Parent: National Energy Investment Group, Zhejiang Provincial Energy Group
# wiki doesn't match spreadsheet, which does have parent percentages

# Benga power station
# Wiki project details section ostensibly is for both units (phase 1 & phase 2),
# but it only lists one sponsor/parent pair, which matches spreadsheet phase 1 (not exact string match, but it is match when human reads it)

# Bielsko-Biala power station
# Wiki has no project details section; no mention of the sponsor for the operating unit (unit EC1)

# Blachownia power station: only retired/cancelled; skip?

# CPI Dabieshan power station
# Project Details for Units 3 & 4 Expansion
# Ownership for units 1 & 2 covered by narrative, but not specifically in project details
# Wiki page doesn't have a row for Parent--but what is entered as sponsor is same as what spreadsheet has for parent

# CPI Pingwei power station
# Phase I: covered in "Plant Details" section, although it's not clear how the 2 sponsors (with no percentages listed) correspond to the 2 parents (with percentages listed)
# doesn't match what's in spreadsheet
# Phase II: not covered in any project details section
# data is included in spreadsheet
# Phase III: covered in section "Project Details for Phase III Expansion"
# sponsor & parent don't match spreadsheet
# Phase IV: covered in section "Project Details for Phase IV Expansion"
# wiki doesn't list sponsor at all, but it is in spreadsheet; for parent, wiki and spreadsheet don't match

# CPI Shanxi Houma power station
# There are 6 retired units; all the ownership information in the wiki page matches what the spreadsheet has for these retired units
# For the 2 operating units, spreadsheet has different sponsor & owner than stated in the wiki



# Duernrohr power station
# Wiki page lists: "Sponsor: Verbund (Unit 1), EVN (Unit 2)"

# Patuakhali power station (Ashuganj)
# Wiki page doesn't agree with sheet; wiki page only lists one set of sponsors, for all units: "Sponsor: Ashuganj Power Station Company Limited (APSCL), China Energy Engineering Corporation"

# Gacko Thermal Power Plant
# Wiki page project details is only for the announced unit (excludes the operating unit)

# Kakanj Thermal Power Plant
# Wiki page has "Project Details for Unit 8" but no project details section for other units

# Tuzla Thermal Power Plant
# Wiki page has section "Project Details for expansion" but other units don't have project details sections

# Porto do Pecém power station: wiki page has all details laid out with references
# Sponsor:
#     Unit 1: EDP Energias do Brasil[2]
#     Unit 2: EDP Energias do Brasil[2]
#     Unit 3: Eneva[14]

# Sihanoukville CEL power station:
# wiki page has project details section that lists details for all units;
# but for sponsor & parent, only has one set of info, which matches the operating unit 1 & 2, but not unit 3



In [ ]:
# TO DO:
# Wabamum Generating Station: Wiki has typo in page name & throughout. Should be "Wabamun" (ending with an "n").
# Angul Steel power station: need to make sponsor names consistent; one has Jindal Steel & Power Ltd and others have Jindal Steel & Power
# Asam-Asam power station: unit names in wiki and spreadsheet don't match up; wiki calls those under construction Unit 5 & 6
# Belchatow power station: sponsor is written two different ways for different units: PGE Górnictwo i Energetyka Konwencjonalna SA & PGE GiEK
# CEEC Chongzuo power station: sponsor is written two different ways; make consistent

In [ ]:
def find_project_details_line_num(one_page):
    project_details_line_num = '' # initialize
    for x in range(len(one_page)):
        one_page_line = one_page[x]
        if one_page_line.startswith('== '):
            one_page_line = one_page_line.replace('== ', '==')
        else:
            pass

        if (one_page_line.strip().lower().startswith('==project details') | 
            one_page_line.strip().lower().startswith('==plant details') | 
            one_page_line.strip().lower().startswith('==plant data')
           ):
            project_details_line_num = x
            break
            # TO DO: handle cases with more than one project/plant details section
            # e.g., "Plant Details" & "Plant Details of expansion"

    return project_details_line_num

In [ ]:
def create_wiki_df(all_pages):
    
    # initialize new df
    df = pd.DataFrame(columns=['page num', 'title', 'category', 'value'])
    
    for page_num in range(len(all_pages)):
        one_page = all_pages[page_num]

        redirect = False # initialize
        for x in range(len(one_page)):
            if '#REDIRECT' in one_page[x]:
                redirect = True

        if redirect == False:
            for x in range(len(one_page)):
                if one_page[x].startswith('    <title>'):
                    title_line_num = x
                    break

            title = one_page[title_line_num].split('<title>')[1].split('</title>')[0]

            if title.startswith('Category'):
                pass
            else:
                project_details_line_num = find_project_details_line_num(one_page)

                if project_details_line_num == '':
                    print(f"No project details section found for {title} (page_num: {page_num})")

                elif project_details_line_num != '':
                    # get line number for next section after project details
                    for x in range(project_details_line_num+1, len(one_page)):
                        if one_page[x].startswith('=='):
                            next_section_line_num = x
                            break

                    # within project details, find first-level entries (one asterisk)
                    for line in range(project_details_line_num+1, next_section_line_num):
                        one_page_line = one_page[line]
                        # remove spaces after asterisks
                        one_page_line = one_page_line.replace('* ', '*')
                        # remove line break at end, and strip
                        one_page_line = one_page_line.replace('\n', '').strip()                    

                        if one_page_line.startswith("*'") and "'''" in one_page_line:
                            line_split = one_page_line.split(":")
                            category = line_split[0].strip("*' ")
                            # print(category)
                            try:
                                value = line_split[1].split('\n')[0].strip("*' ")
                            except:
                                print(f"For {title}, wasn't able to get value from line: {one_page_line}")
                                value = ''
                            # print(value)

                            # TO DO: if a line has the status and capacity in it, parse those

                            # put into df
                            df = df.append(pd.DataFrame({
                                'page num': [page_num],
                                'title': [title],
                                'category': [category],
                                'value': [value]}))

                        elif one_page_line.startswith("'''"):
                            line_split = one_page_line.split(":")
                            category = line_split[0].strip("' ")
                            # print(category)
                            try:
                                value = line_split[1].split('\n')[0].strip("' ")
                            except:
                                print(f"For {title}, wasn't able to get value from line: {one_page_line}")
                                value = ''
                            # print(value)

                            # TO DO: if a line has the status and capacity in it, parse those

                            # put into df
                            df = df.append(pd.DataFrame({
                                'page num': [page_num],
                                'title': [title],
                                'category': [category],
                                'value': [value]}))

                        elif one_page_line.startswith("**"):
                            # second-level entries
                            # for now, don't handle these
                            pass

                        elif one_page_line == '':
                            pass

                        else:
                            # print(f"For {title}, unexpected type of line: {one_page_line}")
                            # TO DO: look more into how to get data out of these
                            pass
                        
    return df

In [ ]:
def clean_wiki_data_df(df):
    df = df.reset_index(drop=True)
    df['value'] = df['value'].str.strip()

    # harmonize terms in df
    # Sponsor
    df['category'] = df['category'].replace({
        'Owner': 'Sponsor',
        'Sponsor/Owner': 'Sponsor',
    })

    # Parent
    df['category'] = df['category'].replace({
        'Parent company': 'Parent',
        'Parent Company': 'Parent',
    })
    # Coal type
    df['category'] = df['category'].replace('Coal Type', 'Coal type')
    
    # Coal source
    df['category'] = df['category'].replace('Coal Source', 'Coal source')
    
    # Start year
    df['category'] = df['category'].replace({
        'Start Year': 'Start year',
        'Start date': 'Start year',
        'Projected in service': 'Start year',
        # typos:
        'Stary year': 'Start year',
        'Star year': 'Start year',
        # in service
        'In service': 'Start year',
        'In Service': 'Start year',
    })
    
    # Retired year
    df['category'] = df['category'].replace('Retired', 'Retired year')
    
    # Capacity
    df['category'] = df['category'].replace('Gross capacity', 'Capacity')
    df['category'] = df['category'].replace('Gross Capacity', 'Capacity')
    
    # Combustion technology (term in spreadsheet)
    df['category'] = df['category'].replace('Type', 'Combustion technology')
    
    return df

In [ ]:
wiki_df = create_wiki_df(all_pages)
wiki_df = clean_wiki_data_df(wiki_df)

in Existing coal plants in China, there are:

Sponsor/owner entries:
* Sponsor: 865
* Owner: 57
* Sponsor/Owner: 15

Parent entries:
* Parent company: 600
* Parent Company: 32
* Parent: 12

In [ ]:
# sponsor_list = ['Sponsor', 'Owner', 'Sponsor/Owner']
# sponsor_values = df[df['category'].isin(sponsor_list)]['value']
# sponsor_values = sponsor_values.str.replace('[', '', regex=False).str.replace(']', '', regex=False)
# sponsor_values.value_counts().head(30)

In [ ]:
def clean_values(sel_val, sel_col):
    """
    Remove parts of values before comparing wiki vs spreadsheet.
    """
    sel_val = sel_val.strip('. ')
    
    if sel_col == 'Status':
        sel_val = sel_val.replace('Canceled', 'Cancelled')
        sel_val = sel_val.replace('canceled', 'Cancelled')

    if sel_col in ['Sponsor', 'Parent']:
        for search_str in [
            'Limited', 'Ltd', 
            'Corporation', 'Corp', 
            'Company', 'Co', 
            'Sa', 'SA'
        ]:
            if sel_val.endswith(search_str):
                sel_val = sel_val[:-len(search_str)]
            
        sel_val = sel_val.strip('. ')
    
    if sel_val == 'nan':
#         print("found a nan!")
        sel_val = ''
    
    return sel_val

In [ ]:
def compare_wiki_and_spreadsheet_plant_level_data(wiki_df, spreadsheet, sel_cols):
    """
    For each plant in spreadsheet, get values in selected column.
    If same for all units in a plant, compare against wiki.
    If not the same for all units in a plant, compare one by one against wiki.
    Find which unit entries match wiki.
    
    Only set up to work for Sponsor, Parent.
    """
    
    cols_handled = ['Sponsor', 'Parent']
    
    sel_col_not_plant_level = 'no' # initialize
    for sel_col in sel_cols:
        if sel_col not in cols_handled:
            sel_col_not_plant_level = 'yes'
            print(f"The selected column is not a plant-level characteristic: {sel_col}")
            print("The function will only process plants with one unit.")
        else:
            pass

    results_df_cols = [
        'Match status',
        'Fuzz partial ratio',
        'Wiki title',
        'sel_col',
        'sel_col - wiki num',
        'sel_col - wiki',
        'sel_col - spreadsheet'
    ]

    results_df = pd.DataFrame(columns=results_df_cols)
     
    if sel_col_not_plant_level == 'yes':
        # filter to only keep plants with a single unit
        units_per_plant = spreadsheet.groupby('Plant')['Unit'].count()
        single_units = units_per_plant[units_per_plant == 1]
        spreadsheet = spreadsheet[spreadsheet['Plant'].isin(single_units.index)]
    else:
        pass
    
    # create list of wiki titles to process
    # exclude template pages; clean up names
    titles_to_process = wiki_df['title'].unique().tolist()    
    titles_to_process = [x for x in titles_to_process if x.endswith('template page') == False]
    titles_to_process = [x.replace('&amp;', '&') for x in titles_to_process]
    print(f"Number of titles to process: {len(titles_to_process)}")

    for title_num in range(len(titles_to_process)):
        if title_num % 1000 == 0:
            print(f"Processing title_num {title_num}")
        else:
            pass

        # select all data for this wiki page
        sel_title = titles_to_process[title_num]
        sel_wiki = wiki_df[wiki_df['title']==sel_title]

        for sel_col in sel_cols:

            # get sel_col listed in one wiki page
            # TO DO: change to ['value'].unique().tolist(); cut line after with list(set(...))
            wiki_sel_col_vals = sel_wiki[sel_wiki['category']==sel_col]['value'].tolist()
            wiki_sel_col_vals = list(set(wiki_sel_col_vals))

            # clean up to remove references (using list comprehension)
            wiki_sel_col_vals = [x.split('&lt;')[0] for x in wiki_sel_col_vals]

            # clean up to remove double brackets (for internal wiki links)
            wiki_sel_col_vals = [x.replace('[', '').replace(']', '') for x in wiki_sel_col_vals]

            # clean up &amp;
            wiki_sel_col_vals = [x.replace('&amp;', '&') for x in wiki_sel_col_vals]

            # get sel_col listed in spreadsheet for this same wiki page
            sheet_sel_col = spreadsheet[spreadsheet['Plant'].str.strip()==sel_title][sel_col]
            # TO DO: change to ['value'].unique().tolist(); cut line after with list(set(...))
            sheet_sel_col_vals = sheet_sel_col.astype(str).tolist()
            sheet_sel_col_vals = list(set(sheet_sel_col_vals))

            if len(sheet_sel_col_vals) == 0:
                data_dict = {
                    'Match status': 'plant not found in spreadsheet',
                    'Wiki title': sel_title,
                    'sel_col': sel_col,
                }
                results_df = results_df.append(data_dict, ignore_index=True)

            else:
                wiki_sel_col_num = 1 # initialize numbering within a given plant
                for wiki_sel_col_val in wiki_sel_col_vals:
                    # compare wiki against spreadsheet
                    for sheet_sel_col_val in sheet_sel_col_vals:

                        # if there is a pipe, keep only the part before the pipe
                        # example: Aliağa Enka power station has "[[Enka Enerji|Enka]]"
                        wiki_sel_col_val = wiki_sel_col_val.split('|')[0]

                        # clean each of the values:
                        wiki_sel_col_val_for_compare = clean_values(wiki_sel_col_val, sel_col)
                        sheet_sel_col_val_for_compare = clean_values(sheet_sel_col_val, sel_col)

                        if wiki_sel_col_val.lower() == sheet_sel_col_val.lower():
                            match_value = 'match'
                        else:
                            if wiki_sel_col_val_for_compare == sheet_sel_col_val_for_compare:
                                match_value = 'close match'
                            else:
                                match_value = 'mismatch'

                        fuzz_partial_ratio = fuzz.partial_ratio(
                            str(wiki_sel_col_val_for_compare).lower(), 
                            str(sheet_sel_col_val_for_compare).lower()
                        )
                        data_dict = {
                            'Match status': match_value,
                            'Fuzz partial ratio': fuzz_partial_ratio,
                            'Wiki title': sel_title,
                            'sel_col': sel_col,
                            'sel_col - wiki num': wiki_sel_col_num,
                            'sel_col - wiki': wiki_sel_col_val,
                            'sel_col - spreadsheet': sheet_sel_col_val,
                        }
                        results_df = results_df.append(data_dict, ignore_index=True)

                    # update wiki_sel_col_num after iterating through sheet_sel_col_vals
                    wiki_sel_col_num += 1

    return results_df

In [ ]:
results_status = compare_wiki_and_spreadsheet_plant_level_data(
    wiki_df=df, 
    spreadsheet=spreadsheet,
    sel_cols=['Status'])

In [ ]:
results_status['Match status'].value_counts()

In [ ]:
results_status['sel_col - spreadsheet'].value_counts()

In [ ]:
wiki_first_word = results_status['sel_col - wiki'].str.split(' ').str[0]
spread_first_word = results_status['sel_col - spreadsheet'].str.split(' ').str[0]
results_status['first word compare'] = wiki_first_word.str.lower()==spread_first_word.str.lower()
results_status['first word compare'].value_counts()

In [ ]:
results_status[
    (results_status['sel_col - spreadsheet'].isna()==False)
    & (results_status['Match status'].isin(['mismatch', 'close match']))
    & (results_status['first word compare']!=True)
]

In [ ]:
status_sel = results_status[
    (results_status['Match status']=='mismatch') &
    (results_status['first word compare']==False)
].sort_values(by='sel_col - wiki')

accepted_statuses_first_word_lower = [
    'operating',
    'mothballed',
    'cancelled',
    'retired',
    'shelved',
    'construction',
    'pre-permit development',
    'announced',
    'permitted',
]

status_sel[~status_sel['sel_col - wiki'].str.split(' ').str[0].str.lower().isin(accepted_statuses_first_word_lower)]

In [ ]:
results_spon_par = compare_wiki_and_spreadsheet_plant_level_data(
    wiki_df=df, 
    spreadsheet=spreadsheet,
    sel_cols=['Sponsor', 'Parent'])

In [ ]:
# # export results
# results_spon_par.to_excel(
#     gcpt_path + 
#     f'GCPT wiki vs spreadsheet comparison for China - {save_timestamp}.xlsx', 
#     index=False)

In [ ]:
parent_check = results_spon_par[results_spon_par['sel_col']=='Parent']
sponsor_check = results_spon_par[results_spon_par['sel_col']=='Sponsor']

In [ ]:
parent_check['Match status'].value_counts()

In [ ]:
low_fuzz = 0
high_fuzz = 70

test = parent_check[
    (parent_check['Match status']=='mismatch') & 
    (parent_check['Fuzz partial ratio'] >= low_fuzz) & 
    (parent_check['Fuzz partial ratio'] <= high_fuzz)
]

missing_values = test[(test['sel_col - wiki']=='') | (test['sel_col - spreadsheet']=='')]
print(f"Number of rows with missing value for either wiki or spreadsheet: {len(missing_values)}")
test = test[~test.index.isin(missing_values.index)]

test = test.sort_values(by='sel_col - spreadsheet')
test = test.reset_index(drop=True)

print(f"Number of rows with data with fuzz partial ratios between {low_fuzz} & {high_fuzz}: {len(test)}")

test.loc[0:50]

In [ ]:
test[test['sel_col - spreadsheet']=='nan']

In [ ]:
# example of a case that's fine, but it looked wrong:
# For CKI Lamma power station, Parents:
# wiki: Power Assets Holdings, Cheung Kong Infrastructure, CK Hutchison Holdings
# spreadsheet: Cheung Kong Infrastructure, CK Hutchison Holdings, Power Assets Holdings
# some same info, different order

In [ ]:
sponsor_check['Match status'].value_counts()

In [ ]:
# results_df[results_df['Match status']=='plant not found in spreadsheet']

In [ ]:
# wiki title CITIC Zhengzhou power station: 
# possibly the same as spreadsheet L100725 (Zhengzhou Xinli power station, other names: Henan Xinli power station, Zhengzhou power station)



In [ ]:
# TO DO: for now, exclude cases with more than one sponsor for a plant (either in the spreadsheet or the wiki)
# Those cases will take extra attention.

In [ ]:
multi_sponsor_wiki = results_df[results_df['Wiki sponsor num'] > 1]

In [ ]:
spreadsheet_sponsor_counts = spreadsheet[['Wiki page', 'Sponsor']].drop_duplicates().groupby('Wiki page')['Sponsor'].count()
multi_sponsor_sheet = spreadsheet_sponsor_counts[spreadsheet_sponsor_counts > 1]

In [ ]:
multi_sponsor_sheet.sort_values()

In [ ]:
sel_wiki = 'https://www.gem.wiki/Banten_Suralaya_power_station'
spreadsheet[spreadsheet['Wiki page']==sel_wiki][['Plant', 'Unit', 'Sponsor']].sort_values(by='Unit')

In [ ]:
# Notes:
# Yuzhnouralsk power station: 
# Looks like sponsors counted up based on dragging to fill cells.
# But also see wiki. Not sure they should all be OGK-3.
# There's also GRES-2 Expansion (cancelled), with another sponsor. But this doesn't seem to be in the spreadsheet.



In [ ]:
results_df[results_df['Wiki title']=='Hebi Fenghe power station']

In [ ]:
results_df[(results_df['Fuzz partial ratio'] < 90) & 
           (results_df['Fuzz partial ratio'] > 80)]

In [ ]:
spreadsheet[spreadsheet['Plant']=='Anhui Bengbu power station']['Sponsor'].unique().tolist()

In [ ]:
results_df['Match status'].value_counts()

In [ ]:
results_df

In [ ]:
type_values = df[df['category']=='Type']['value']
type_values = type_values.str.replace('[', '').str.replace(']', '')
type_values = type_values.str.lower()
type_values.value_counts().head(30)

# TO DO: parse, splitting on '),' and ');'
# Then take units sections, generate lines for each unit (to compare against spreadsheet)


## Existing coal plants in China:
* Anhui Hwasu power station has: Plant Details & Plant Details of expansion
* Bengbu City power station: like the Baima power station page, but not labeled as a stub
* Binzhou Heating Supply Center power station: Has section "Details of Heating Supply Center power station" with the usual Project Details stuff
* Zhejiang Jiaxing Ultra-supercritical Power Generation Project: Has a fair amount of information, but no project details (or equivalent) section
* Huadian Huangjiaozhuang power station: Has some info, but no project details (or equivalent) section
* Neijiang Baima power station: has section "Unit 4 & 5 Project Details"
* Huaneng Dalian power station: Has some info, but no project details (or equivalent) section
* Huaneng Xilinhot power station: Has some info, but no project details (or equivalent) section
* Shanqiu Fengyuan Aluminum power station: had section "Plantt Details"; edited page to fix typo
* Baima power station: stub
* Dezhou power station: stub
* Green Energy power station: stub
* Tuoketuo-1 power station: stub
* Yuzhou power station: stub
* Huangen Fuzhou power station: stub
* Qinbei power station: stub
* Qitaihe power station: stub

In [ ]:
df[df['category'].str.contains('capacity')]

# TO DO: 
* for each page, get the line number for Project Details section
  * will look like: "\n==Project Details==\n"
* then get line number for next section:
  * will look like: "\n=="
* for just the Project Details section:
  * parse to get a list of all the row names: anything that starts with *

In [ ]:
for line in data:
    if line.startswith('    <title>'):
        # create new list to add to
        page_list = []
        
        # put everything into the list until hitting another line that starts with '    <title>'
        